In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

import toolbox
import fcn
import yaml
import shutil
import sys

import chainer

%matplotlib inline
sns.set()

%load_ext autoreload
# Note: this reload all lib at each cell exec, just for convenience
%autoreload 2 

chainer.print_runtime_info()

In [ ]:
%%time
from models import InstanceSeg, SemanticSeg, PanopticSeg

print("Preparing instance segmentation model...", end=" ")
instaseg = InstanceSeg(
    "./2018-12-03_23-24/20181203_232507/params.yaml",
    "./2018-12-03_23-24/20181203_232507/snapshot_model_45000.npz",
    gpu=0,
)
print("Done")

print("Preparing semantic segmentation model...", end=" ")
semaseg = SemanticSeg(
    "./toolbox/deeplab/config/cocostuff164k.yaml",
    "./cocostuff164k_iter100k.pth",
    gpu=0,
)
print("Done")

print("Preparing panotpic model...", end=" ")
panoseg = PanopticSeg(instaseg, semaseg, thresh=0.7, frac=0.2)
print("Done")

In [ ]:
# Create a minival dataset, to quickly confirm things work
dest_name = "minival"
MINIVAL_SIZE = 100

root = Path("./dataset/annotations/panoptic_val2017/")
r_labels = np.random.choice(list(root.glob("*.png")), size=MINIVAL_SIZE, replace=False)
r_images = [f.parent.parent.parent.joinpath(
    "val2017").joinpath("{}.jpg".format(f.stem)) for f in r_labels]

r_labels[0].parent.parent.joinpath(dest_name).mkdir(exist_ok=True, parents=True)
r_images[0].parent.parent.joinpath(dest_name).mkdir(exist_ok=True, parents=True)

cp_r_labels = [f.parent.parent.joinpath(dest_name).joinpath(f.name) for f in r_labels]
cp_r_images = [f.parent.parent.joinpath(dest_name).joinpath(f.name) for f in r_images]

r_labels[0], cp_r_labels[0], r_images[0], cp_r_images[0]

import shutil
for src, dst in zip(r_labels, cp_r_labels):
    shutil.copyfile(src, dst)
    
for src, dst in zip(r_images, cp_r_images):
    shutil.copyfile(src, dst)

In [ ]:
from tqdm import tqdm
import warnings
import json

def predict_and_save(img, img_id, img_dir, tmp_json):    
    segment, RGB = panoseg.predict(img, img_id)
    
    skimage.io.imsave(img_dir.joinpath(segment["file_name"]), RGB)
    with open(tmp_json.joinpath("{}.json".format(img_id)), "w") as f:
        json.dump(segment, f)

# Make predictions for all test set.
# Takes a while (~30h), so to make it more practical
# it can be interrupted and continued.

out_path = Path("./panopticapi/sample_data/")
split_name = "panoptic_test2017_drcnn"

# Prepare directories for intermediate results
img_dir = out_path.joinpath(split_name)
img_dir.mkdir(parents=True, exist_ok=True)
tmp_json = out_path.joinpath("{}_separated_json".format(split_name))
tmp_json.mkdir(parents=True, exist_ok=True)

# Input images
in_path = Path("./dataset/test2017/")
imgs = list(in_path.glob("*.jpg"))

errs = []

_json_cache = out_path.joinpath("{}_separated_json".format(split_name))
for ip in tqdm(imgs):
    id_img = int(ip.stem.lstrip("0"))
    
    if _json_cache.joinpath("{}.json".format(id_img)).exists():
        continue
    try:
        img = skimage.io.imread(str(ip))
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            predict_and_save(img, id_img, img_dir, tmp_json)
    except Exception as e:
        errs.append({"image": str(ip), "error": str(e)})

if len(errs) > 0:
    with open(out_path.joinpath("{}_error_log.json".format(split_name)), "w") as f:
        json.dump(errs, f)

In [ ]:
# Combine all predictions

buff = []
for j in tmp_json.glob("*.json"):
    with j.open() as f:
        buff.append(json.load(f))

with open(out_path.joinpath("{}.json".format(split_name)), "w") as f:
    json.dump({"annotations": buff}, f)

# shutil.rmtree(tmp_json)